Получение различных массивов `terms` в зависимости от частоты их встречаемости.  
Веса в IA.txt: самые редкие `term` имеют больший вес, но не все `term` из  
IA.txt присутсвуют в train_terms.

In [2]:
from pathlib import Path
import pandas as pd


# Корень проекта.
DIR_ROOT = Path.cwd().parent.parent
# Путь к удаленной директории с ресурсами: данные, модели и т.д.
DIR_REMOTE: Path | None = Path('/home/admin/cafa/resources')

if DIR_REMOTE is not None and DIR_REMOTE.exists():
    DIR_RESOURCE = DIR_REMOTE
else:
    DIR_RESOURCE = DIR_ROOT

In [5]:
train_terms = pd.read_csv(DIR_RESOURCE / 'data/raw/Train/train_terms.tsv', sep="\t")

train_terms.shape

(5363863, 3)

In [8]:
ia_df = pd.read_csv(DIR_RESOURCE / 'data/raw/IA.txt', sep="\t", header=None)
ia_df.columns = ['term', 'weight']
ia_df.head()

,term,weight
0,GO:0000001,0.000000
1,GO:0000002,3.103836
2,GO:0000003,3.439404
3,GO:0000011,0.056584
4,GO:0000012,6.400377


In [13]:
# добавляем веса из ia_df в таблицу с исходными данными

trains_terms_with_weights = pd.merge(train_terms, ia_df, on='term', how='left')
trains_terms_with_weights.head()

,EntryID,term,aspect,weight
0,A0A009IHW8,GO:0008152,BPO,1.598544
1,A0A009IHW8,GO:0034655,BPO,0.042274
2,A0A009IHW8,GO:0072523,BPO,0.083901
3,A0A009IHW8,GO:0044270,BPO,0.281155
4,A0A009IHW8,GO:0006753,BPO,0.013844


In [12]:
# убираем дубликаты `term`, сортируем вес по убыванию
trains_terms_with_weights = trains_terms_with_weights.drop_duplicates(
    subset=['term']).sort_values(by=['weight'], ascending=False
)
# сохраняем в список 1500 самых редких == с наибольшим весом `term`
labels_1500_rare = trains_terms_with_weights['term'].head(1500).to_list()
len(labels_1500_rare)

1500

In [11]:
# обновляем дата сет оставляя в нем только те белки, у которых были редкие
# онтологии

train_terms_rare = train_terms.loc[train_terms['term'].isin(labels_1500_rare)]
train_terms_rare.head()

,EntryID,term,aspect
554,A0A024B7I3,GO:2000290,BPO
2213,A0A060XXJ7,GO:0090664,BPO
2974,A0A061I403,GO:0044601,BPO
5032,A0A088FLK9,GO:0019835,BPO
5566,A0A096MJY4,GO:0060290,BPO


In [14]:
train_terms_rare.shape

(11563, 3)

In [ ]:
# Загрузка id белков, используемых далее для определения embeddings.
train_protein_ids = np.load(
    DIR_RESOURCE / 'data/interim/t5embeds/train_ids.npy'
)
print(train_protein_ids.shape)

In [ ]:
train_embeddings = np.load(
    DIR_RESOURCE / 'data/interim/t5embeds/train_embeds.npy'
)

# Трансформация embeddings в `pandas` датафрейм.
column_num = train_embeddings.shape[1]
train_features_df = pd.DataFrame(
    train_embeddings,
    columns = ["Column_" + str(i) for i in range(1, column_num+1)]
)
# Удаление неиспользуемых в дальнейшем объектов.
del train_embeddings
print(train_features_df.shape)
print(round(train_features_df.memory_usage().sum() / 1024**3, 2), 'GB')

In [ ]:
# Формирование нулевой матрицы с необходимыми размерами:
# например train_size x num_of_labels (142246 x 1500)
num_of_labels = 1500

train_size = train_protein_ids.shape[0]
train_labels = np.zeros((train_size, num_of_labels))

# Конвертация `numpy` одномерного массива в `pandas.Series`.
series_train_protein_ids = pd.Series(train_protein_ids)

# Проход по каждому уникальному GO-терму.
for i in range(num_of_labels):
    # Для каждого label чтение сответсвующих данных `train_terms`
    n_train_terms = train_terms_rare[
        train_terms_rare['term'] == labels_1500_rare[i]
    ]

    # Выбор подмножества с уникальными EntryId (названиями белков) для
    # текущей цели (GO-терм). Не уникальность связана с 3-мя типами аспектов.
    label_related_proteins = n_train_terms['EntryID'].unique()

    # В серии `series_train_protein_ids` pandas, если белок связан
    # с текущей целью, то ей присваивается 1, иначе 0
    # (через конвертацию bool в float).
    #  Замена i-го столбеца в `train_labels` на эту серию.
    train_labels[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)

# Конвертация матрицы из numpy в pandas датафрейм.
labels_df = pd.DataFrame(data = train_labels, columns = labels)
labels_df = labels_df.astype('int32')
print(labels_df.shape)
print(round(labels_df.memory_usage().sum() / 1024**3, 2), 'GB')

In [ ]:
# # Сохранение признаков.
# train_features_df.to_csv(
#     DIR_RESOURCE / 'data/prepared/train_features.csv',
#     index=False,
# )
# # Сохранение целей.
# labels_df.to_csv(
#     DIR_RESOURCE / 'data/prepared/train_lbls_top1500_goterms.csv',
#     index=False,
# )